In [3]:
import sys
sys.path.append("..")

In [4]:
import pandas as pd
import gymnasium as Env

In [5]:
%load_ext autoreload
%autoreload 2

from src.environments.fx_environment import FxTradingEnv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
FX_data = pd.read_parquet("../data/FX_data.parquet.gzip")

Todo:

Create ABC

Tests:

1. Unknown currency in initial portfolio
2. Missing currency in initial portfolio
3. Portfolio value in base
4. Portfolio weights == 1

In [7]:
current_porfolio = {
    "usd": 100_000,
    "eur": 100_000,
    # "jpy": 100_000,
    "sgd": 100_000,
}

In [8]:
fx_env = FxTradingEnv(
    initial_portfolio=current_porfolio,
    historical_prices=FX_data,
)

In [9]:
fx_env.preprocess_data()

jpy not in inital portfolio. Setting quantity to 0


In [10]:
fx_env.current_portfolio

{'usd': 100000, 'eur': 100000, 'sgd': 100000, 'jpy': 0.0}

In [11]:
fx_env._convert_portfolio_to_base_ccy()

{'usd': 100000.0, 'eur': 144300.0, 'sgd': 71515.41157119359, 'jpy': 0.0}

In [12]:
fx_env.current_portfolio_value

315815.4115711936

In [13]:
fx_env.current_portfolio_weights

{'usd': 0.3166406588661909,
 'eur': 0.4569124707439135,
 'sgd': 0.22644687038989553,
 'jpy': 0.0}